### 1) 파이토치를 이용한 텍스트 분류(Text classification using PyTorch)

In [16]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets
import random

In [3]:
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

In [6]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

cpu와 cuda 중 다음 기기로 학습함: cuda


###### (1) torchtext를 이용한 전처리

In [18]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
# 순차적인 데이터셋임을 알 수 있도록 sequential 인자값으로 True를 명시
LABEL = data.Field(sequential=False, batch_first=True)

###### (2) 데이터 로드 및 분할하기

In [20]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 59.2MB/s]


In [21]:
print('trainset의 구성 요소 출력 : ', trainset.fields)

trainset의 구성 요소 출력 :  {'text': <torchtext.legacy.data.field.Field object at 0x7f89c5cde190>, 'label': <torchtext.legacy.data.field.Field object at 0x7f89c5cdefd0>}


In [22]:
print('testset의 구성 요소 출력 : ', testset.fields)

testset의 구성 요소 출력 :  {'text': <torchtext.legacy.data.field.Field object at 0x7f89c5cde190>, 'label': <torchtext.legacy.data.field.Field object at 0x7f89c5cdefd0>}


In [23]:
print(vars(trainset[0]))

{'text': ['as', 'a', 'disclaimer,', "i've", 'seen', 'the', 'movie', '5-6', 'times', 'in', 'the', 'last', '15', 'years,', 'and', 'i', 'only', 'just', 'saw', 'the', 'musical', 'this', 'week.', 'this', 'allowed', 'me', 'to', 'judge', 'the', 'movie', 'without', 'being', 'tainted', 'by', 'what', 'was', 'or', "wasn't", 'in', 'the', 'musical', '(however,', 'it', 'tainted', 'me', 'when', 'i', 'watched', 'the', 'musical', ':)', ')', '<br', '/><br', '/>i', 'actually', 'believe', 'michael', 'douglas', 'worked', 'quite', 'well', 'in', 'that', 'role,', 'along', 'with', 'kasey.', 'i', 'think', 'her', "'let", 'me', 'dance', 'for', 'you', "scene'", 'is', 'one', 'of', 'the', 'best', 'parts', 'of', 'the', 'movie,', 'a', 'worthwhile', 'addition', 'compared', 'to', 'the', 'musical.', 'the', 'dancers', 'and', 'singing', 'in', 'the', 'movie', 'are', 'much', 'superior', 'to', 'the', 'musical,', 'as', 'well', 'as', 'the', 'cast', 'which', 'is', 'at', 'least', '10', 'times', 'bigger', '(easier', 'to', 'do', 'i

###### (3) 데이터 집합 만들기

In [24]:
TEXT.build_vocab(trainset, min_freq=5) # 단어 집합 생성
LABEL.build_vocab(trainset)

In [25]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 46159
클래스의 개수 : 2


###### (4) 데이터 로더 만들기

In [26]:
trainset, valset = trainset.split(split_ratio=0.8)

In [27]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

In [28]:
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))

훈련 데이터의 미니 배치의 개수 : 313
테스트 데이터의 미니 배치의 개수 : 391
검증 데이터의 미니 배치의 개수 : 79


In [30]:
batch = next(iter(train_iter)) # 첫번째 미니배치
print(batch.text.shape)

torch.Size([64, 988])


In [31]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

###### (5) RNN 모델 구현하기

In [47]:
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [48]:
model = GRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

###### (1) tain (모델 함수 만듦)

In [49]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

In [50]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [51]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

[Epoch: 1] val loss :  0.70 | val accuracy : 50.24
[Epoch: 2] val loss :  0.69 | val accuracy : 52.02
[Epoch: 3] val loss :  0.69 | val accuracy : 53.04
[Epoch: 4] val loss :  0.70 | val accuracy : 52.46
[Epoch: 5] val loss :  0.60 | val accuracy : 70.92
[Epoch: 6] val loss :  0.38 | val accuracy : 82.60
[Epoch: 7] val loss :  0.32 | val accuracy : 86.46
[Epoch: 8] val loss :  0.32 | val accuracy : 87.30
[Epoch: 9] val loss :  0.33 | val accuracy : 87.26
[Epoch: 10] val loss :  0.35 | val accuracy : 86.50


In [52]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.34 | 테스트 정확도: 85.07
